In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import textblob
import gensim
import spacy
import os

import data

Takes a 5 minutes the first time, but use `use_cached` afterward so you don't have to wait in the future.

In [4]:
train, train_vecs, test, test_vecs = data.get_data(use_cached=True)

Using Cached


In [3]:
stuff = data._data()

/Users/ryan/miniconda3/envs/chatty/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
/Users/ryan/miniconda3/envs/chatty/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [7]:
a, b = data._make_train_test_split(stuff)

In [5]:
train.shape

(95047, 9)

# Goals of Exploration

- Use word2vec as implemented in gensim
- Classify the Sentiment
- Classify dialogue action
- Predict the next sentiment from:
    - Previous sentiments
        - including both persons
    - Previous dialogue actions
        - including both persons

Should put this into a module so we don't take up so much memory

In [4]:
LAG_MASK = ['act', 'emo', 'polarity', 'subjectivity']

train_lags = data.get_lags(train[LAG_MASK], lag_range=[1, 2])

In [12]:
train.head(1)

,,person,utter,act,emo,polarity,subjectivity,topic,biggest_drawup,biggest_drawdown
conv,,,,,,,,,,
0,0,person_a,The kitchen stinks .,directive,disgust,-0.6,0.5,ordinary_life,0.0,0.0


In [164]:
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.neural_network import MLPClassifier

act = train['act']
ACT_MASK = act.notnull()

assert act[act.notnull()].shape[0] == train_vecs[act.notnull()].shape[0]

In [ ]:
StratifiedShuffleSplit(n_splits=5)

In [160]:
nn = MLPClassifier((10, ))
nn.fit(train_vecs[ACT_MASK], act[ACT_MASK])

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [167]:
nn.predict_proba(train_vecs[100:101])

array([[0.24201223, 0.30767956, 0.35729817, 0.09301004]])

In [168]:
act[100:101]

conv   
22    0    question
Name: act, dtype: object